# Function calling: Assistants

## 1\. 환경 설정

In [3]:
# 필요한 라이브러리를 설치한다.
%pip install -q finance-datareader

In [ ]:
import os
from dotenv import load_dotenv  

!echo "OPENAI_API_KEY=<Open AI 발급한 Key를 입력하세요." >> .env #최초만 설정하여 실행
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

MODEL="gpt-4.1-mini"

In [11]:
import json
import datetime
import FinanceDataReader as fdr
from openai import OpenAI

client = OpenAI()

## 2\. Tools (함수) 정의

모델이 사용할 수 있는 함수 목록을 `tools` 필드에 정의한다. 

각 함수는 **이름(name)**, **설명(description)**, 그리고 JSON Schema 형식의 \*\*파라미터(parameters)\*\*로 구성된다. 

이 명세를 통해 모델은 각 함수의 용도와 필요한 인자를 이해하게 된다.

In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_data",
            "description": "특정 종목의 지정된 날짜의 주가나 거래량을 조회한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "조회할 종목 코드(ticker). 예: '035420' (네이버)",
                    },
                    "date": {
                        "type": "string",
                        "description": "조회할 날짜. 'YYYY-MM-DD' 형식이다.",
                    },
                    "result_type": {
                        "type": "string",
                        "enum": ["price", "volume"],
                        "description": "조회할 정보의 종류. 'price' 또는 'volume'이다.",
                    }
                },
                "required": ["ticker", "date", "result_type"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "order_stock",
            "description": "주식을 시장가 또는 지정가로 매수/매도 주문을 넣는다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "주문할 종목 코드(ticker). 예: 'NVDA' (엔비디아)",
                    },
                    "action_type": {
                        "type": "string",
                        "enum": ["매수", "매도"],
                        "description": "주문 종류. '매수' 또는 '매도'이다.",
                    },
                    "shares": {
                        "type": "integer",
                        "description": "주문할 주식의 수.",
                    },
                    "order_type": {
                        "type": "string",
                        "enum": ["시장가", "지정가"],
                        "description": "주문 방식. '시장가' 또는 '지정가'이다.",
                    },
                    "order_price": {
                        "type": "number",
                        "description": "지정가 주문일 경우의 목표 가격.",
                    }
                },
                "required": ["ticker", "action_type", "shares", "order_type"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_currency",
            "description": "한 화폐를 다른 화폐로 환율에 맞게 변환한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "from_currency": {
                        "type": "string",
                        "enum": ["KRW", "USD", "EUR", "JPY"],
                        "description": "변환의 기준이 되는 화폐.",
                    },
                    "to_currency": {
                        "type": "string",
                        "enum": ["KRW", "USD", "EUR", "JPY"],
                        "description": "변환하고자 하는 목표 화폐.",
                    },
                    "amount": {
                        "type": "number",
                        "description": "변환할 금액 (from_currency 기준).",
                    },
                    "date": {
                        "type": "string",
                        "description": "환율 기준 날짜. 'YYYY-MM-DD' 형식이다.",
                    }
                },
                "required": ["from_currency", "to_currency", "amount", "date"]
            }
        }
    }
]

## 3\. 사용자 함수 구현

`tools`에 정의한 함수들의 실제 로직을 파이썬 코드로 구현한다. 각 함수는 API 호출의 결과로 받은 인자들을 사용하여 동작한다.

In [13]:
def get_stock_data(ticker, date, result_type):
    """FinanceDataReader를 사용해 주식 데이터를 조회한다."""
    try:
        stock_data = fdr.DataReader(ticker, date)
        if stock_data.empty:
            return json.dumps({"error": f"{ticker}에 대한 {date} 데이터가 없다."})

        row = stock_data.loc[date] if date in stock_data.index else stock_data.iloc[0]

        if result_type == "price":
            return json.dumps({"date": date, "price": float(row['Close'])})
        elif result_type == "volume":
            return json.dumps({"date": date, "volume": int(row['Volume'])})
        else:
            return json.dumps({"error": "잘못된 조회 유형이다."})
    except Exception as e:
        return json.dumps({"error": str(e)})


def order_stock(ticker, action_type, shares, order_type, order_price=None):
    """주식 주문을 시뮬레이션하고 결과를 반환한다."""
    if order_type == "지정가" and (order_price is None or order_price <= 0):
        return json.dumps({"error": "지정가 주문에는 유효한 가격이 필요하다."})

    # 실제 환경에서는 이 부분에 DB 연동 또는 증권사 API 호출 로직이 들어간다.
    return json.dumps({"status": "success", "message": f"{ticker} 종목 {shares}주 {order_type}로 '{action_type}' 주문이 완료되었다."})


def calculate_currency(from_currency, to_currency, amount, date):
    """가상의 고정 환율을 사용하여 금액을 변환한다."""
    # 실제 환경에서는 외부 환율 API를 호출하여 최신 정보를 가져와야 한다.
    exchange_rates = {
        "KRW": {"USD": 0.00073, "EUR": 0.00068, "JPY": 0.11},
        "USD": {"KRW": 1370.50, "EUR": 0.93, "JPY": 157.33},
        "EUR": {"KRW": 1475.25, "USD": 1.08, "JPY": 169.50},
        "JPY": {"KRW": 8.71, "USD": 0.0064, "EUR": 0.0059},
    }
    rate = exchange_rates.get(from_currency, {}).get(to_currency)
    if not rate:
        return json.dumps({"error": "지원되지 않는 통화다."})

    converted_amount = round(amount * rate, 2)
    return json.dumps({"converted_amount": f"{converted_amount} {to_currency}"})


def process_tool_calls(tool_call):
    """모델이 요청한 함수를 이름에 따라 실행하고 결과를 반환한다."""
    tool_map = {
        "get_stock_data": get_stock_data,
        "order_stock": order_stock,
        "calculate_currency": calculate_currency
    }

    func_name = tool_call.function.name
    func_to_call = tool_map.get(func_name)

    if not func_to_call:
        raise ValueError(f"알 수 없는 함수 이름: {func_name}")

    func_args = json.loads(tool_call.function.arguments)

    return func_to_call(**func_args)

## 4\. 실행 및 테스트

이제 전체 실행 과정을 테스트한다. 사용자 질문을 받아 모델에 전달하고, 모델이 함수 호출을 요청하면 해당 함수를 실행한 뒤, 그 결과를 다시 모델에게 보내 최종 답변을 생성하도록 한다.

### 4.1 단일 함수 호출

In [16]:
# 사용자 질문을 정의한다.
query = "네이버 어제 날짜 주가 알려줘"

# 날짜를 동적으로 계산한다.
yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
processed_query = query.replace("어제 날짜", yesterday)

messages = [{"role": "user", "content": processed_query}]
print(f"User: {processed_query}")

# 1단계: 모델에 메시지와 함수 정보를 전달한다.
first_response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response_message = first_response.choices[0].message
tool_calls = response_message.tool_calls

# 2단계: 모델이 함수 호출을 요청했는지 확인한다.
if not tool_calls:
    print(f"Assistant: {response_message.content}")
else:
    # 3단계: 요청된 함수를 실행한다.
    messages.append(response_message)  # 어시스턴트의 응답(함수 호출 요청)을 대화에 추가한다.

    for tool_call in tool_calls:
        print(f"Assistant 요청: {tool_call.function.name}({tool_call.function.arguments})")
        function_response = process_tool_calls(tool_call)
        print(f"Tool 실행 결과: {function_response}")

        # 4단계: 함수 실행 결과를 대화에 추가한다.
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": tool_call.function.name,
                "content": function_response,
            }
        )

    # 5단계: 함수 실행 결과를 포함하여 모델에 다시 요청해 최종 답변을 받는다.
    second_response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )

    final_answer = second_response.choices[0].message.content
    print(f"Assistant: {final_answer}")

User: 네이버 2025-08-11 주가 알려줘
Assistant 요청: get_stock_data({"ticker":"035420","date":"2025-08-11","result_type":"price"})
Tool 실행 결과: {"date": "2025-08-11", "price": 223000.0}
Assistant: 네이버(035420)의 2025년 8월 11일 주가는 223,000원 입니다.


### 4.2 병렬 함수 호출

OpenAI API는 하나의 요청에 대해 여러 함수를 동시에 호출하는 **병렬 함수 호출**을 기본적으로 지원한다. 아래 코드는 `get_stock_data`와 `order_stock`을 동시에 호출하는 예시다.

In [17]:
# 사용자 질문
query = "오늘 테슬라 주가 얼마야? 그리고 시장가로 20주 매수 주문해줘."

# 날짜를 동적으로 계산한다.
today = datetime.datetime.now().strftime('%Y-%m-%d')
processed_query = query.replace("오늘", today)

messages = [{"role": "user", "content": processed_query}]
print(f"User: {processed_query}")

# 1단계: 모델에 메시지와 함수 정보를 전달한다.
first_response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
response_message = first_response.choices[0].message
tool_calls = response_message.tool_calls

# 2단계: 모델의 함수 호출 요청을 처리한다.
if tool_calls:
    messages.append(response_message)

    # 3단계: 모든 함수 호출을 실행한다.
    for tool_call in tool_calls:
        print(f"Assistant 요청: {tool_call.function.name}({tool_call.function.arguments})")
        function_response = process_tool_calls(tool_call)
        print(f"Tool 실행 결과: {function_response}")

        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": tool_call.function.name,
                "content": function_response,
            }
        )

    # 4단계: 모든 함수 실행 결과를 포함하여 최종 답변을 요청한다.
    second_response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    final_answer = second_response.choices[0].message.content
    print(f"Assistant: {final_answer}")

User: 2025-08-12 테슬라 주가 얼마야? 그리고 시장가로 20주 매수 주문해줘.
